# Part B: Finite Element Music Synthesis

## Importing the libraries

In [1]:
import os
nvvm_lib_path = !find / -iname 'libnvvm.so'
dev_lib_path = !find / -iname 'libdevice'
assert len(nvvm_lib_path)>0, "NVVM Missing"
assert len(dev_lib_path)>0, "Device Lib Missing"
os.environ['NUMBAPRO_NVVM'] = nvvm_lib_path[0]
os.environ['NUMBAPRO_LIBDEVICE'] = dev_lib_path[0]

In [2]:
import cv2
import matplotlib.pyplot as plt
import math
import numpy as np
import numba
from numba import cuda

/usr/local/lib/python3.6/dist-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/usr/local/lib/python3.6/dist-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda-10.1/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


## Part 1: Implementing grid Sequentially

In [3]:
def implement_sequentially(N, iterations):
    u = np.zeros([N, N], dtype = float) # displacement of element
    
    u1 = u.copy() # displacement at previous time step
    u2 = u.copy() # displacement at previous previous time step
    
    #Constants
    n = 0.0002
    p = 0.5
    G = 0.75 # boundary gain

    u1[2][2] = 1 #initialize
    for iterator in range(iterations):
        for i in range(1, N-1):
            
            for j in range(1, N-1):
              # Interior element operation
                u[i][j] = (
                    (p * (u1[i-1][j] + u1[i+1][j] + u1[i][j-1] + u1[i][j+1] - 4*u1[i][j])) + 
                           (2 * u1[i][j]) - ((1 - n) * u2[i][j])
                           )
                u[i][j] = u[i][j]/(1+n)

        for iter2 in range(1, N-1):
            # Updating side elements
            u[0][iter2] = G * u[1][iter2] # right side
            u[N-1][iter2] = G * u[N-2][iter2] # left side
            u[iter2][0] = G * u[iter2][1] # top side
            u[iter2][N-1] = G * u[iter2][N-2] # bottom side

        # Updating corner elements
        u[0][0] = G * u[1][0] # top left
        u[N-1][0] = G * u[N-2][0] # top right
        u[0][N-1] = G * u[0][N-2] # bottom left
        u[N-1][N-1] = G * u[N-1][N-2] # bottom right

        #Re-assigning u
        u2 = u1.copy()
        u1 = u.copy()

        print("u[2, 2] at iteration {}: {}".format(iterator, u[2, 2]))

implement_sequentially(4, 4) 

u[2, 2] at iteration 0: 0.0
u[2, 2] at iteration 1: -0.49980001999999923
u[2, 2] at iteration 2: 2.9982007234403314e-08
u[2, 2] at iteration 3: 0.28102511495301846


## Part 2: Single Thread Parallelization

In [13]:
# Constants

n = 0.0002
p = 0.5
G = 0.75

# Grid config
g_size = 4
grid = np.zeros((g_size, g_size)) # new grid
grid_1 = grid.copy()
grid_2 = grid.copy()
grid_1[2][2] = 1   # initialize

# Parameters
num_of_blocks = 978
numThreads_perBlock = 1000
iterations_T = 50


@cuda.jit
def parallel_interior(grid, grid_1, grid_2, n, p):
  
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < 16:
    j = idx // 4
    i = idx % 4
    
    # Interior node operation
    if i not in [0, 3] and j not in [0, 3]:
      
      grid[i][j] = ((p * (grid_1[i-1][j] + grid_1[i+1][j] + grid_1[i][j-1] + grid_1[i][j+1] - 4 * grid_1[i][j])) + 
                    (2 * grid_1[i][j]) - ((1 - n) * grid_2[i][j]))
      grid[i][j] = grid[i][j]/(1 + n)

@cuda.jit
def parallel_sides(grid, G):
  
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < 16:
    j = idx // 4
    i = idx % 4
    
    # Updating sides
    if i == 0 and j in [1, 2]:
      grid[i][j] = G * grid[i + 1][j] # left side
    elif i == 3 and j in [1, 2]:
      grid[i][j] = G * grid[i - 1][j]  # right side
    elif j == 0 and i in [1, 2]:
      grid[i][j] = G * grid[i][j + 1] # top side
    elif j == 3 and i in [1, 2]:
      grid[i][j] = G * grid[i][j - 1] # bottom side

@cuda.jit
def parallel_corners(grid, G):
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < 16:
    j = idx // 4
    i = idx % 4

    # Updating corners
    if i == 0 and j == 0:
      grid[i][j] = G * grid[i + 1][j] # top left corner
    elif i == 0 and j == 3:
      grid[i][j] = G * grid[i][j - 1] # bottom left corner
    elif i == 3 and j == 0:
      grid[i][j] = G * grid[i][j + 1] # top right corner
    elif i == 3 and j == 3:
      grid[i][j] = G * grid[i][j - 1] # bottom right corner


# executing parallelization
for iteration in range(iterations_T):
  parallel_interior[num_of_blocks, numThreads_perBlock](grid, grid_1, grid_2, n, p)
  parallel_sides[num_of_blocks, numThreads_perBlock](grid, G)
  parallel_corners[num_of_blocks, numThreads_perBlock](grid, G)
  
  print("u[2, 2] at iteration {}: {}".format(iteration, grid[2][2]))

  grid_2 = grid_1.copy()
  grid_1 = grid.copy()

u[2, 2] at iteration 0: 0.0
u[2, 2] at iteration 1: -0.49980001999999923
u[2, 2] at iteration 2: 2.9982007234403314e-08
u[2, 2] at iteration 3: 0.2810251149530184
u[2, 2] at iteration 4: 0.04682818495502672
u[2, 2] at iteration 5: -0.08778516995465212
u[2, 2] at iteration 6: -0.32181476130245723
u[2, 2] at iteration 7: -0.74136664956306
u[2, 2] at iteration 8: -0.38839948920730266
u[2, 2] at iteration 9: 0.6652254330040654
u[2, 2] at iteration 10: 0.7787261426358246
u[2, 2] at iteration 11: -0.22371278398803726
u[2, 2] at iteration 12: -0.5139857688941788
u[2, 2] at iteration 13: 0.3315687356910853
u[2, 2] at iteration 14: 0.7216416509417185
u[2, 2] at iteration 15: 0.16843621887136329
u[2, 2] at iteration 16: -0.28548607266696807
u[2, 2] at iteration 17: -0.4078320015272444
u[2, 2] at iteration 18: -0.563128497465155
u[2, 2] at iteration 19: -0.2835166141928366
u[2, 2] at iteration 20: 0.4472320260386376
u[2, 2] at iteration 21: 0.39596970515004054
u[2, 2] at iteration 22: -0.49652669

## Part 3 - Multiple Block Parallelization

In [21]:
# Constants
G = 0.75
n = 0.0002
p = 0.5
g_size = 512

# Grid config
grid = np.zeros((g_size, g_size))
grid_1 = grid.copy()
grid_2 = grid.copy()
grid_1[g_size // 2][g_size // 2] = 1   # initialize

# Parameters
iterations_T = 50
num_of_blocks = 978
numThreads_perBlock = 1000
elements_per_thread = 16

@cuda.jit
def multi_interior(grid, grid_1, grid_2, n, p, g_size, elements_in_thread, needed_threads):
  
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < needed_threads:
    
    for iter in range(elements_in_thread):
    
      this_Idx = idx * elements_in_thread + iter
      # Keeping track of previous grids
      if this_Idx < g_size * g_size:
        j = this_Idx // g_size
        i = this_Idx % g_size
        if i != 0 and j != 0 and i != g_size - 1 and j != g_size - 1:
          # Interior node operation
          grid[i][j] = (
              (p * (grid_1[i-1][j] + grid_1[i+1][j] + grid_1[i][j-1] + grid_1[i][j+1] - 4 * grid_1[i][j])) + 
                        (2 * grid_1[i][j]) - ((1 - n) * grid_2[i][j]))
          grid[i][j] = grid[i][j]/(1 + n)

@cuda.jit
def multi_sides(grid, G, g_size, elements_in_thread, needed_threads):
  
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < needed_threads:
    
    for iter in range(elements_in_thread):
      # Keeping track of previous grids
      this_Idx = idx * elements_in_thread + iter

      if this_Idx < g_size * g_size:
        j = this_Idx // g_size
        i = this_Idx % g_size

        # Updating sides
        if i == 0 and j in [1, 2]:
          grid[i][j] = G * grid[i + 1][j] # left side
        elif i == 3 and j in [1, 2]:
          grid[i][j] = G * grid[i - 1][j] # right side
        elif j == 0 and i in [1, 2]:
          grid[i][j] = G * grid[i][j + 1] # top side
        elif j == 3 and i in [1, 2]:
          grid[i][j] = G * grid[i][j - 1] # bottom side

@cuda.jit
def multi_corners(grid, G, g_size, elements_in_thread, needed_threads):
  idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

  if idx < needed_threads:

    for iter in range(elements_in_thread):
      # updating on adjacent nodes isn't a problem because we're using the history grids only
      this_Idx = idx * elements_in_thread + iter

      if this_Idx < g_size * g_size:
        j = this_Idx // g_size
        i = this_Idx % g_size
        # Updating corners
        if i == 0 and j == 0:
          grid[i][j] = G * grid[i + 1][j] # top left corner
        elif i == 0 and j == 3:
          grid[i][j] = G * grid[i][j - 1] # bottom left corner
        elif i == 3 and j == 0:
          grid[i][j] = G * grid[i][j + 1] # top right corner
        elif i == 3 and j == 3:
          grid[i][j] = G * grid[i][j - 1] # bottom right corner


# executing multi parallelization
needed_threads = math.ceil((g_size * g_size) / elements_per_thread)
for iter in range(iterations_T):
  multi_interior[num_of_blocks, numThreads_perBlock](grid, grid_1, grid_2, n, p, g_size, elements_per_thread, needed_threads)
  multi_sides[num_of_blocks, numThreads_perBlock](grid, G, g_size, elements_per_thread, needed_threads)
  multi_corners[num_of_blocks, numThreads_perBlock](grid, G, g_size, elements_per_thread, needed_threads)
  
  print("Iteration {}: {}".format(iter, grid[g_size // 2][g_size // 2]))

  grid_2 = grid_1.copy()
  grid_1 = grid.copy()

Iteration 0: 0.0
Iteration 1: 3.998400479824981e-08
Iteration 2: 0.0
Iteration 3: 0.24980013992803154
Iteration 4: 0.0
Iteration 5: 8.9892080370116e-08
Iteration 6: 0.0
Iteration 7: 0.14040029222121936
Iteration 8: 0.0
Iteration 9: 1.403440891682952e-07
Iteration 10: 0.0
Iteration 11: 0.09742231969133452
Iteration 12: 0.0
Iteration 13: 1.9087120709162718e-07
Iteration 14: 0.0
Iteration 15: 0.0745294056393437
Iteration 16: 0.0
Iteration 17: 2.413783011458448e-07
Iteration 18: 0.0
Iteration 19: 0.060320634512427584
Iteration 20: 0.0
Iteration 21: 2.918343471751079e-07
Iteration 22: 0.0
Iteration 23: 0.050645649253699805
Iteration 24: 0.0
Iteration 25: 3.4222637009984303e-07
Iteration 26: 0.0
Iteration 27: 0.0436341257299501
Iteration 28: 0.0
Iteration 29: 3.925480441346873e-07
Iteration 30: 0.0
Iteration 31: 0.03831973287361857
Iteration 32: 0.0
Iteration 33: 4.4279595196121775e-07
Iteration 34: 0.0
Iteration 35: 0.03415301791490642
Iteration 36: 0.0
Iteration 37: 4.929681108493421e-07
I

## Comparing Results
Checking if results in Part 3 are the same for Part 1 and 2

In [27]:
# Constant
n = 0.0002
p = 0.5
G = 0.75

# Grid config
g_size = 4
grid = np.zeros((g_size, g_size))
grid_1 = grid.copy()
grid_2 = grid.copy()
grid_1[g_size // 2][g_size // 2] = 1   # initialize

# variable parameters
num_of_blocks = 978
numThreads_perBlock = 1000
elements_per_thread = 16
iterations_T = 50

needed_threads = math.ceil((g_size * g_size) / elements_per_thread)

# executing multi paralellization
for iter in range(iterations_T):
  multi_interior[num_of_blocks, numThreads_perBlock](grid, grid_1, grid_2, n, p, g_size, elements_per_thread, needed_threads)
  multi_sides[num_of_blocks, numThreads_perBlock](grid, G, g_size, elements_per_thread, needed_threads)
  multi_corners[num_of_blocks, numThreads_perBlock](grid, G, g_size, elements_per_thread, needed_threads)
  
  print("Iteration {}: {}".format(iter, grid[g_size // 2][g_size // 2]))

  grid_2 = grid_1.copy()
  grid_1 = grid.copy()

Iteration 0: 0.0
Iteration 1: -0.49980001999999923
Iteration 2: 2.9982007234403314e-08
Iteration 3: 0.2810251149530184
Iteration 4: 0.04682818495502672
Iteration 5: -0.08778516995465212
Iteration 6: -0.32181476130245723
Iteration 7: -0.74136664956306
Iteration 8: -0.38839948920730266
Iteration 9: 0.6652254330040654
Iteration 10: 0.7787261426358246
Iteration 11: -0.22371278398803726
Iteration 12: -0.5139857688941788
Iteration 13: 0.3315687356910853
Iteration 14: 0.7216416509417185
Iteration 15: 0.16843621887136329
Iteration 16: -0.28548607266696807
Iteration 17: -0.4078320015272444
Iteration 18: -0.563128497465155
Iteration 19: -0.2835166141928366
Iteration 20: 0.4472320260386376
Iteration 21: 0.39596970515004054
Iteration 22: -0.496526695683186
Iteration 23: -0.5256373939774961
Iteration 24: 0.5760366497416338
Iteration 25: 0.9926034468428319
Iteration 26: 0.1908710314349021
Iteration 27: -0.4025014078396819
Iteration 28: -0.27847620339368123
Iteration 29: -0.1845074875605142
Iteration

Evidently from the above results of Part 3, we can confirm that they are the same as those of Part 1 and 2. Hence, the multi paralellization was done correctly.

## Speed-up Data

In [29]:
import time

# Constant
n = 0.0002
p = 0.5
G = 0.75

# grid creation
g_size = 512
grid = np.zeros((g_size, g_size))
grid_1 = grid.copy()
grid_2 = grid.copy()
grid_1[g_size // 2][g_size // 2] = 1   # initialize

iterations_T = 500

for num_of_blocks in [1, 4, 8, 32, 128, 512, 2048]:
  
  for numThreads_perBlock in [1, 4, 16, 64, 256, 1024]:
    
    for elements_per_thread in [1, 4, 16, 64, 256]:
      start = time.time()
      needed_threads = math.ceil((g_size * g_size) / elements_per_thread)

      # executing multi paralellization
      for iter in range(iterations_T):
        drum_interiorMult[num_of_blocks, numThreads_perBlock](grid, grid_1, grid_2, n, p, g_size, elements_per_thread, needed_threads)
        drum_edgesMult[num_of_blocks, numThreads_perBlock](grid, G, g_size, elements_per_thread, needed_threads)
        drum_cornersMult[num_of_blocks, numThreads_perBlock](grid, G, g_size, elements_per_thread, needed_threads)

        grid_2 = grid_1.copy()
        grid_1 = grid.copy()
      
      print("Runtime for {} blocks, {} threads, {} elements: {}".format(num_of_blocks, numThreads_perBlock, elements_per_thread, time.time() - start))

Runtime for 1 blocks, 1 threads, 1 elements: 4.560439825057983
Runtime for 1 blocks, 1 threads, 4 elements: 4.587965250015259
Runtime for 1 blocks, 1 threads, 16 elements: 4.549238920211792
Runtime for 1 blocks, 1 threads, 64 elements: 4.614914178848267
Runtime for 1 blocks, 1 threads, 256 elements: 4.8032026290893555
Runtime for 1 blocks, 4 threads, 1 elements: 4.589372396469116
Runtime for 1 blocks, 4 threads, 4 elements: 4.562153577804565
Runtime for 1 blocks, 4 threads, 16 elements: 4.586965084075928
Runtime for 1 blocks, 4 threads, 64 elements: 4.559335231781006
Runtime for 1 blocks, 4 threads, 256 elements: 5.088643312454224
Runtime for 1 blocks, 16 threads, 1 elements: 4.554551124572754
Runtime for 1 blocks, 16 threads, 4 elements: 4.570990800857544
Runtime for 1 blocks, 16 threads, 16 elements: 4.593151807785034
Runtime for 1 blocks, 16 threads, 64 elements: 4.652085781097412
Runtime for 1 blocks, 16 threads, 256 elements: 5.200481414794922
Runtime for 1 blocks, 64 threads, 1 e